Splits inputs into into training and test and normalizes (from and to /work/milesep/convective_outlook_ml)

In [6]:
import xarray as xr
import numpy as np

In [7]:
detail = 'slgt_small'

In [8]:
if detail == 'slgt_small':
    ds = xr.open_zarr("/glade/work/milesep/convective_outlook_ml/inputs_raw_" + detail + "_cds.zarr")
else:
    ds = xr.open_zarr("/glade/work/milesep/convective_outlook_ml/inputs_raw_" + detail + ".zarr")

In [9]:
ds.load()

<xarray.Dataset> Size: 4GB
Dimensions:    (latitude: 26, longitude: 60, day: 4071, tod: 4, level: 5)
Coordinates:
  * day        (day) datetime64[ns] 33kB 2002-04-02 2002-04-03 ... 2023-12-17
  * latitude   (latitude) float64 208B 50.0 49.0 48.0 47.0 ... 27.0 26.0 25.0
  * level      (level) float64 40B 925.0 850.0 700.0 500.0 300.0
  * longitude  (longitude) float64 480B -125.0 -124.0 -123.0 ... -67.0 -66.0
  * tod        (tod) int64 32B 0 6 12 18
Data variables: (12/13)
    d2m        (latitude, longitude, day, tod) float32 102MB 273.7 ... 289.9
    pv         (level, latitude, longitude, day, tod) float32 508MB -3.004e-0...
    q          (level, latitude, longitude, day, tod) float32 508MB 0.002201 ...
    t          (level, latitude, longitude, day, tod) float32 508MB 275.5 ......
    t2m        (latitude, longitude, day, tod) float32 102MB 284.0 ... 298.1
    tisr       (latitude, longitude, day, tod) float32 102MB 2.469e+06 ... 3....
    ...         ...
    u10        (latitude, longitude, day, tod) float32 102MB 0.6641 ... -12.03
    v          (level, latitude, longitude, day, tod) float32 508MB -1.556 .....
    v10        (latitude, longitude, day, tod) float32 102MB -0.873 ... 3.966
    w          (level, latitude, longitude, day, tod) float32 508MB 0.3545 .....
    z          (level, latitude, longitude, day, tod) float32 508MB 8.299e+03...
    z_sfc      (latitude, longitude, day, tod) float32 102MB 670.3 ... 6.417
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2025-08-17T05:24 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [10]:
def estimate_dataset_size_bytes(ds):
    total_bytes = 0
    for var in ds.data_vars.values():
        if var.chunks is not None:
            total_bytes += var.nbytes
        else:
            # Use 64-bit integers to avoid overflow
            n_elements = np.prod(var.shape, dtype=np.int64)
            dtype_size = np.dtype(var.dtype).itemsize
            total_bytes += int(n_elements * dtype_size)
    return total_bytes


size_bytes = estimate_dataset_size_bytes(ds)
print(f"Estimated uncompressed size: {size_bytes / 1e9:.2f} GB")


Estimated uncompressed size: 4.17 GB


In [11]:
# split into training and test datasets

train_ds = ds.sel(day=slice('2002-01-01', '2019-12-31'))
test_ds = ds.sel(day=slice('2020-01-01', '2024-12-31'))
test_ds = test_ds.chunk({'day': 10})

In [12]:
# mean and std for each day
def compute_daily_stats(ds):
    day_stats = {}
    for var in ds.data_vars:
        dims = ds[var].dims

        # Determine whether level is in the variable
        reduce_dims = set(dims) & {'latitude', 'longitude', 'tod'}
        group_dims = ['day']
        if 'level' in dims:
            group_dims.append('level')

        # Compute stats
        mean = ds[var].groupby('day').mean(dim=list(reduce_dims | {'tod'}), skipna=True)
        std = ds[var].groupby('day').std(dim=list(reduce_dims | {'tod'}), skipna=True)

        # Preserve level as dimension
        mean = mean.rename(f'{var}_mean')
        std = std.rename(f'{var}_std')

        day_stats[f'{var}_mean'] = mean
        day_stats[f'{var}_std'] = std

    return xr.Dataset(day_stats)


# Compute the stats
daily_stats_ds = compute_daily_stats(train_ds)

In [13]:
daily_stats_ds

<xarray.Dataset> Size: 1MB
Dimensions:     (day: 3414, level: 5)
Coordinates:
  * day         (day) datetime64[ns] 27kB 2002-04-02 2002-04-03 ... 2019-12-22
  * level       (level) float64 40B 925.0 850.0 700.0 500.0 300.0
Data variables: (12/26)
    d2m_mean    (day) float32 14kB 276.3 275.7 274.7 275.6 ... 275.4 272.7 274.0
    d2m_std     (day) float32 14kB 10.46 11.75 9.535 8.643 ... 11.4 12.5 9.427
    pv_mean     (level, day) float32 68kB 8.638e-07 1.025e-06 ... 5.083e-07
    pv_std      (level, day) float32 68kB 3.199e-06 3.533e-06 ... 9.302e-07
    q_mean      (level, day) float32 68kB 0.004659 0.004823 ... 0.0001645
    q_std       (level, day) float32 68kB 0.00307 0.003411 ... 0.0001366
    ...          ...
    w_mean      (level, day) float32 68kB 0.009222 -0.004788 ... -0.02362
    w_std       (level, day) float32 68kB 0.2256 0.2138 0.2364 ... 0.183 0.2768
    z_mean      (level, day) float32 68kB 7.605e+03 7.87e+03 ... 9.148e+04
    z_std       (level, day) float32 68kB 507.9 475.7 ... 3.085e+03 1.869e+03
    z_sfc_mean  (day) float32 14kB 5.124e+03 5.124e+03 ... 5.124e+03 5.124e+03
    z_sfc_std   (day) float32 14kB 6.415e+03 6.415e+03 ... 6.415e+03 6.415e+03

In [15]:
daily_stats_ds = daily_stats_ds.compute()
daily_stats_ds.to_netcdf("data/processed_data/daily_input_stats_" + detail + ".nc")

In [17]:
def compute_overall_from_daily_stats(daily_stats_ds):

    overall_stats = {}

    for var in daily_stats_ds.data_vars:
        if var.endswith("_mean"):
            base_name = var[:-5]
            mean_name = f"{base_name}_mean"
            std_name = f"{base_name}_std"

            mean_da = daily_stats_ds[mean_name]
            std_da = daily_stats_ds[std_name]

            # Mean of daily means
            overall_mean = mean_da.mean(dim="day", skipna=True)

            # Pooled std calculation:
            # std_total = sqrt( mean(std_i^2 + (mu_i - mu_total)^2) ) (law of total variance)
            variance_component = std_da**2 + (mean_da - overall_mean)**2
            overall_std = (variance_component.mean(dim="day", skipna=True))**0.5

            overall_stats[f"{base_name}_mean"] = overall_mean
            overall_stats[f"{base_name}_std"] = overall_std

    return xr.Dataset(overall_stats)

In [18]:
def standardize_with_stats(ds, stats):
    standardized = {}
    for var in ds.data_vars:
        da = ds[var]
        mean = stats[f"{var}_mean"]
        std = stats[f"{var}_std"]
        standardized[var] = (da - mean) / std
    return xr.Dataset(standardized)

In [19]:
train_stats = compute_overall_from_daily_stats(daily_stats_ds)

In [20]:
train_ds_std = standardize_with_stats(train_ds, train_stats)
test_ds_std = standardize_with_stats(test_ds, train_stats)

In [21]:
train_ds_std.to_zarr("/glade/work/milesep/convective_outlook_ml/train_inputs_" + detail + ".zarr", mode="w")

In [22]:
test_ds_std.to_zarr("/glade/work/milesep/convective_outlook_ml/test_inputs_" + detail + ".zarr", mode="w")